Contentbased recommendation v_1

Opis: Za svaki film napravimo vektor(item vector) sa žanrovima(postoji dvadeset žanrova) gdje je vrijednost stupca za neki žanr
    jednaka jedan(true) ako film pripada nekom žanru, nula(false) ako ne. Potom za svakog korisnika prosumiramo vektore
    filmova koje je on pogledao i podijelimo s brojem pogledanih filmova. Time dobivamo user vektor. Potom iteriramo
    kroz sve filmove i tražimo najsličnije filmove(cosine similarity) odnosno item vektore našeg user vektora.
    Korisniku potom predlažemo petnaes najsličnijh filmova njegovom user vektoru.


In [11]:
import pandas as pd
import numpy as np
import itertools

ratings = pd.read_csv("ml-latest-small/ratings.csv")
movies = pd.read_csv("ml-latest-small/movies.csv")

movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [ ]:
ratings.head()

U ovom dijelu iz csv-a(comma separated values) izvlačimo žanrove jer su oni svi napisani kao jedna vrijednost
odvojena znakom |. I pravimo pandas dataframe. Rezultat na kraju je dataframe sa filmovima u obliku item vektora i
još nadodan movieId i title.

In [14]:
values = movies['genres'].str.split("|")

column_names = []
column_names_user = []
for v in values:
    if (len(column_names) == 20):
        break;
    for i in v:
        if i not in column_names:
            column_names.append(i)

column_names_user = column_names.copy()
column_names_user.append('userId')
column_names.append('title')
column_names.append('movieId')
df2 = pd.DataFrame(columns=column_names)

for index, row in movies.iterrows():
    tmp = row['genres'].split("|")
    df2.loc[index, 'title'] = movies.loc[index, 'title']
    df2.loc[index, 'movieId'] = movies.loc[index, 'movieId']
    for i in tmp:
        df2.loc[index, i] = 1
df2 = df2.fillna(0)
data = pd.merge(ratings, df2, on=['movieId'])
df2Helper = df2.drop(columns=['title', 'movieId'])

df2.head()

,Adventure,Animation,Children,Comedy,Fantasy,Romance,Drama,Action,Crime,Thriller,...,Sci-Fi,War,Musical,Documentary,IMAX,Western,Film-Noir,(no genres listed),title,movieId
0,1,1,1,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,Toy Story (1995),1
1,1,0,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,Jumanji (1995),2
2,0,0,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,Grumpier Old Men (1995),3
3,0,0,0,1,0,1,1,0,0,0,...,0,0,0,0,0,0,0,0,Waiting to Exhale (1995),4
4,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,Father of the Bride Part II (1995),5


Sada definiramo funkciju koja za određenog korisnika određuje 15 preporučenih filmova na temelju cosine similartiy-a.

In [15]:
def findBestForUser(n):
    user = data.query("userId == {}".format(n))
    user = user.drop(columns=['title', 'userId', 'movieId', 'rating', 'timestamp'])
    user = np.array(user.mean())
    dictionary = {}
    print("User number: {}".format(n))
    for index, row in df2.iterrows():
        film = np.array(df2Helper.loc[index, :])
        result = user.dot(film) / (np.linalg.norm(user) * np.linalg.norm(film))
        dictionary[df2.loc[index, 'title']] = result

    sorted_dict = {}
    sorted_keys = sorted(dictionary, key=dictionary.get)
    sorted_keys = reversed(sorted_keys)
    for w in sorted_keys:
        sorted_dict[w] = dictionary[w]

    x = itertools.islice(sorted_dict.items(), 0, 15)
    for key, value in x:
        print("     {} : {}".format(key, value))
        
for i in range(5):
    findBestForUser(i + 1)

User number: 1
     Dragonheart 2: A New Beginning (2000) : 0.8670756622462975
     Hunting Party, The (2007) : 0.8455296197390959
     Stunt Man, The (1980) : 0.8245322302201942
     The Great Train Robbery (1978) : 0.8233372412682535
     Flashback (1990) : 0.8233372412682535
     Maximum Ride (2016) : 0.810351086211493
     Extreme Days (2001) : 0.8088657695442655
     Machete (2010) : 0.7944871492561584
     Diamond Arm, The (Brilliantovaya ruka) (1968) : 0.7944871492561584
     After the Sunset (2004) : 0.7944871492561584
     Charlie's Angels: Full Throttle (2003) : 0.7944871492561584
     Super Mario Bros. (1993) : 0.7840146759096195
     Who Am I? (Wo shi shei) (1998) : 0.7833909600207372
     Casanova (2005) : 0.7811717221736529
     Chase, The (1994) : 0.7779370427630333
User number: 2
     Last Boy Scout, The (1991) : 0.9185823775684783
     Wasabi (2001) : 0.9185823775684783
     Another 48 Hrs. (1990) : 0.9185823775684783
     Metro (1997) : 0.9185823775684783
     Bad Boy